In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
raw_df = pd.read_csv('/content/drive/MyDrive/IR_Project/2019-Oct.csv')

In [ ]:
raw_df.head()

In [ ]:
df_dated = raw_df[raw_df['event_time'].str.contains('2019-10-01') | 
          raw_df['event_time'].str.contains('2019-10-02') | 
          raw_df['event_time'].str.contains('2019-10-03') | 
          raw_df['event_time'].str.contains('2019-10-04') | 
          raw_df['event_time'].str.contains('2019-10-05')]

In [ ]:
df_dated = df_dated.fillna('')

In [ ]:
df_categ = df_dated[df_dated['category_code'].str.contains('computers.')]

In [ ]:
interaction_map = {'view':2,'cart':30,'purchase':80}

In [ ]:
df_categ['user_score'] = df_categ['event_type'].map(interaction_map)
df_categ['user_purchase'] = df_categ['event_type'].apply(lambda x: 1 if x=='purchase' else 0)

In [ ]:
categs = df_categ['category_code'].unique()

In [ ]:
df_quart = df_categ

In [ ]:
# categorizing prices into 5-quartiles
df_quart.loc[:, 'price_category'] = 0
price_series = list()
for i in range(len(categs)):
  serie = df_quart[df_quart['category_code'] == categs[i]]['price']
  price_series.append(serie)

In [ ]:
qcut_series = []

In [ ]:
for i in range(len(categs)):
  qcut_serie = pd.qcut(price_series[i], q=5, labels=[1,2,3,4,5])
  qcut_series.append(qcut_serie.tolist())

In [ ]:
qcut_series

In [ ]:
df = df_quart

In [ ]:
df['category_code'].unique()

array(['computers.notebook', 'computers.desktop',
       'computers.peripherals.monitor', 'computers.components.videocards',
       'computers.peripherals.keyboard',
       'computers.components.motherboard', 'computers.peripherals.mouse',
       'computers.peripherals.printer', 'computers.components.memory',
       'computers.ebooks', 'computers.components.power_supply',
       'computers.components.cpu', 'computers.components.hdd',
       'computers.components.cooler', 'computers.peripherals.camera'],
      dtype=object)

In [ ]:
for idx, element in enumerate(df['category_code'].unique()):
  print(len(df.loc[df['category_code']==element, 'price_category']))
  print(len(qcut_series[idx]))

In [ ]:
for idx, element in enumerate(df['category_code'].unique()):
  df.loc[df['category_code']==element, 'price_category'] = qcut_series[idx]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
user_prod_grp = []

In [ ]:
user_prod_grp = df.groupby(['user_id','product_id'])['user_score','user_purchase'].sum().reset_index()

In [ ]:
user_prod_grp['user_purchase'] = user_prod_grp['user_purchase'].apply(lambda x: 1 if x>1 else x)

In [ ]:
std = MinMaxScaler(feature_range=(0.025, 1))
user_prod_grp['user_score'] = user_prod_grp['user_score'].apply(lambda x: 100 if x>100 else x)

In [ ]:
std.fit(user_prod_grp['user_score'].values.reshape(-1,1))
user_prod_grp['interaction_score'] = std.transform(user_prod_grp['user_score'].values.reshape(-1,1))

In [ ]:
user_prod_grp = user_prod_grp.merge(df[['product_id','category_code','brand','price','price_category']].drop_duplicates('product_id'),on=['product_id'])

In [ ]:
df.to_csv("/content/drive/MyDrive/IR_Project/df_fin_v0.csv")
user_prod_grp.to_csv("/content/drive/MyDrive/IR_Project/user_prod_grp_fin_v0.csv")